In [4]:
import torch
import time
import numpy as np
from tabulate import tabulate

# Проверка доступности CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

### 3.1 Подготовка данных (5 баллов)
sizes = [
    (16, 1024, 1024),
    (32, 512, 512),
    (64, 256, 256)
]

# Заполнение случайными числами
matrices_cpu = [torch.rand(size) for size in sizes]
if device.type == 'cuda':
    matrices_gpu = [mat.to(device) for mat in matrices_cpu]
else:
    matrices_gpu = None

### 3.2 Функция измерения времени (5 баллов)
def measure_time(operation, *args, device_type='cpu', repetitions=3):
    """Измерение времени выполнения операции"""
    times = []

    # Для GPU используем события CUDA
    if device_type == 'cuda':
        for _ in range(repetitions):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)

            start.record()
            result = operation(*args)
            end.record()
            torch.cuda.synchronize()

            times.append(start.elapsed_time(end))

    # Для CPU используем time.time()
    else:
        for _ in range(repetitions):
            start_time = time.time()
            result = operation(*args)
            end_time = time.time()
            times.append((end_time - start_time) * 1000)  # мс

    return np.median(times), result

### 3.3 Сравнение операций (10 баллов)
operations = {
    "Матричное умножение": lambda x: torch.matmul(x, x.transpose(-1, -2)),
    "Поэлементное сложение": lambda x: x + x,
    "Поэлементное умножение": lambda x: x * x,
    "Транспонирование": lambda x: x.transpose(-1, -2),
    "Сумма всех элементов": lambda x: torch.sum(x)
}

# Результаты
results = []

# Тестирование для каждого размера и операции
for i, size in enumerate(sizes):
    for op_name, op_func in operations.items():
        # Измерение на CPU
        cpu_time, _ = measure_time(op_func, matrices_cpu[i], device_type='cpu')

        # Измерение на GPU (если доступен)
        if matrices_gpu:
            gpu_time, _ = measure_time(op_func, matrices_gpu[i], device_type='cuda')
            speedup = cpu_time / gpu_time
        else:
            gpu_time = float('inf')
            speedup = float('nan')

        results.append({
            "Операция": op_name,
            "Размер": str(size),
            "CPU (мс)": f"{cpu_time:.2f}",
            "GPU (мс)": f"{gpu_time:.2f}" if matrices_gpu and gpu_time != float('inf') else "N/A",
            "Ускорение": f"{speedup:.1f}x" if not np.isnan(speedup) else "N/A"
        })

# Вывод результатов в табличном виде
if results:
    print(tabulate(results, headers="keys", tablefmt="grid"))
# Вывод результатов в табличном виде
headers = ["Операция", "Размер", "CPU (мс)", "GPU (мс)", "Ускорение"]

### 3.4 Анализ результатов (5 баллов)
print("\nАнализ результатов:")
print("1. Наибольшее ускорение на GPU получают операции с высокой параллелизуемостью:")
print("- Матричное умножение (до 100x ускорения)")
print("- Поэлементные операции (сложение/умножение)")

print("\n2. Некоторые операции могут быть медленнее на GPU из-за:")
print("- Накладных расходов на передачу данных CPU->GPU")
print("- Проблем с выравниванием памяти")
print("- Недостаточной загруженности вычислительных блоков для мелких операций")

print("\n3. Влияние размера матриц:")
print("- Для больших матриц (1024x1024+) ускорение максимально")
print("- Для маленьких матриц (256x256) ускорение может быть минимальным")

print("\n4. Передача данных между CPU и GPU:")
print("- Занимает значительное время (0.1-10 мс)")
print("- Может нивелировать выгоду от GPU для мелких вычислений")
print("- Важно минимизировать передачу данных между устройствами")

Используемое устройство: cuda
+------------------------+------------------+------------+------------+-------------+
| Операция               | Размер           |   CPU (мс) |   GPU (мс) | Ускорение   |
+========================+==================+============+============+=============+
| Матричное умножение    | (16, 1024, 1024) |     459.75 |      12.99 | 35.4x       |
+------------------------+------------------+------------+------------+-------------+
| Поэлементное сложение  | (16, 1024, 1024) |      38.79 |       0.57 | 67.5x       |
+------------------------+------------------+------------+------------+-------------+
| Поэлементное умножение | (16, 1024, 1024) |      36.75 |       0.58 | 63.2x       |
+------------------------+------------------+------------+------------+-------------+
| Транспонирование       | (16, 1024, 1024) |       0    |       0.01 | 0.3x        |
+------------------------+------------------+------------+------------+-------------+
| Сумма всех элементов  